# Python API Example - Access Slots Import and Storage in Dataframe

This guide is designed to provide an example of how to access the Spark API:
- The path to your client credentials is the only input needed to run this script (just before Section 2)
- This script has been designed to display the raw outputs of requests from the API
- This script can be copied and pasted by customers for quick use of the API

__N.B. This guide is just for Access terminal slots data. If you're looking for other API data products (such as contract prices, Freight routes or Netbacks), please refer to their according code example files.__ 

### Have any questions?

If you have any questions regarding our API, or need help accessing specific datasets, please contact us at:

__data@sparkcommodities.com__

or refer to our API website for more information about this endpoint: https://www.sparkcommodities.com/api/request/access.html

## 1. Importing Data

Here we define the functions that allow us to retrieve the valid credentials to access the Spark API.

This section can remain unchanged for most Spark API users.

In [1]:
# import libraries for importing data
import json
import os
import sys
import pandas as pd
from base64 import b64encode
from urllib.parse import urljoin
from pprint import pprint
import requests
from io import StringIO
import time

try:
    from urllib import request, parse
    from urllib.error import HTTPError
except ImportError:
    raise RuntimeError("Python 3 required")


In [2]:
# defining functions 
API_BASE_URL = "https://api.sparkcommodities.com"


def retrieve_credentials(file_path=None):
    """
    Find credentials either by reading the client_credentials file or reading
    environment variables
    """
    if file_path is None:
        client_id = os.getenv("SPARK_CLIENT_ID")
        client_secret = os.getenv("SPARK_CLIENT_SECRET")
        if not client_id or not client_secret:
            raise RuntimeError(
                "SPARK_CLIENT_ID and SPARK_CLIENT_SECRET environment vars required"
            )
    else:
        # Parse the file
        if not os.path.isfile(file_path):
            raise RuntimeError("The file {} doesn't exist".format(file_path))

        with open(file_path) as fp:
            lines = [l.replace("\n", "") for l in fp.readlines()]

        if lines[0] in ("clientId,clientSecret", "client_id,client_secret"):
            client_id, client_secret = lines[1].split(",")
        else:
            print("First line read: '{}'".format(lines[0]))
            raise RuntimeError(
                "The specified file {} doesn't look like to be a Spark API client "
                "credentials file".format(file_path)
            )

    print(">>>> Found credentials!")
    print(
        ">>>> Client_id={}****, client_secret={}****".format(
            client_id[:5], client_secret[:5]
        )
    )

    return client_id, client_secret

def do_api_post_query(uri, body, headers):
    """
    OAuth2 authentication requires a POST request with client credentials before accessing the API. 
    This POST request will return an Access Token which will be used for the API GET request.
    """
    url = urljoin(API_BASE_URL, uri)

    data = json.dumps(body).encode("utf-8")

    # HTTP POST request
    req = request.Request(url, data=data, headers=headers)
    try:
        response = request.urlopen(req)
    except HTTPError as e:
        print("HTTP Error: ", e.code)
        print(e.read())
        sys.exit(1)

    resp_content = response.read()

    # The server must return HTTP 201. Raise an error if this is not the case
    assert response.status == 201, resp_content

    # The server returned a JSON response
    content = json.loads(resp_content)

    return content

def get_access_token(client_id, client_secret):
    """
    Get a new access_token. Access tokens are the thing that applications use to make
    API requests. Access tokens must be kept confidential in storage.

    # Procedure:

    Do a POST query with `grantType` and `scopes` in the body. A basic authorization
    HTTP header is required. The "Basic" HTTP authentication scheme is defined in
    RFC 7617, which transmits credentials as `clientId:clientSecret` pairs, encoded
    using base64.
    """

    # Note: for the sake of this example, we choose to use the Python urllib from the
    # standard lib. One should consider using https://requests.readthedocs.io/

    payload = "{}:{}".format(client_id, client_secret).encode()
    headers = {
        "Authorization": b64encode(payload).decode(),
        "Accept": "application/json",
        "Content-Type": "application/json",
    }
    body = {
        "grantType": "clientCredentials",
        "scopes": "read:access",
    }

    content = do_api_post_query(uri="/oauth/token/", body=body, headers=headers)

    print(
        ">>>> Successfully fetched an access token {}****, valid {} seconds.".format(
            content["accessToken"][:5], content["expiresIn"]
        )
    )

    return content["accessToken"]

## N.B. Credentials

Here we call the above functions, and input the file path to our credentials.

N.B. You must have downloaded your client credentials CSV file before proceeding. Please refer to the API documentation if you have not dowloaded them already.  Instructions for downloading your credentials can be found here:

https://api.sparkcommodities.com/redoc#section/Authentication/Create-an-Oauth2-Client


In [3]:
# Insert file path to your client credentials here
client_id, client_secret = retrieve_credentials(file_path="/tmp/client_credentials.csv")

# Authenticate:
access_token = get_access_token(client_id, client_secret)

>>>> Found credentials!
>>>> Client_id=875f4****, client_secret=6cdf8****
>>>> Successfully fetched an access token eyJhb****, valid 604799 seconds.


## 2. Latest Slot Release

Here we call the latest slot release and print it in a readable format. This is done using the URL:

__/beta/terminal-slots/releases/latest/__


We then save the entire dataset as a local variable called `latest`.

In [4]:
# Function to get the latest slot release
def get_latest_slots():
    uri = urljoin(API_BASE_URL,'/beta/terminal-slots/releases/latest/')
    headers = {
            "Authorization": "Bearer {}".format(access_token),
            "accept": "text/csv"
        }
    response = requests.get(uri, headers=headers)
    if response.status_code == 200:
        df = response.content.decode('utf-8')
        df = pd.read_csv(StringIO(df))
    else:
        print('Bad Request')
    return df

In [5]:
# Call latest slots function
latest = get_latest_slots()
latest.head()

,ReleaseDate,TerminalCode,TerminalName,Total,M+0,M+1,M+2,M+3,M+4,M+5,...,M+4StartDate,M+5StartDate,M+6StartDate,M+7StartDate,M+8StartDate,M+9StartDate,M+10StartDate,M+11StartDate,M+12StartDate,M>12StartDate
0,2025-01-14,adriatic,Adriatic,0,0,0,0,0,0,0,...,2025-05-01,2025-06-01,2025-07-01,2025-08-01,2025-09-01,2025-10-01,2025-11-01,2025-12-01,2026-01-01,2026-02-01
1,2025-01-14,brunsbuttel,Brunsbuttel,0,0,0,0,0,0,0,...,2025-05-01,2025-06-01,2025-07-01,2025-08-01,2025-09-01,2025-10-01,2025-11-01,2025-12-01,2026-01-01,2026-02-01
2,2025-01-14,deutsche-ostsee,Deutsche Ostsee,59,2,5,6,6,4,3,...,2025-05-01,2025-06-01,2025-07-01,2025-08-01,2025-09-01,2025-10-01,2025-11-01,2025-12-01,2026-01-01,2026-02-01
3,2025-01-14,dragon,Dragon,0,0,0,0,0,0,0,...,2025-05-01,2025-06-01,2025-07-01,2025-08-01,2025-09-01,2025-10-01,2025-11-01,2025-12-01,2026-01-01,2026-02-01
4,2025-01-14,dunkerque,Dunkerque,1,1,0,0,0,0,0,...,2025-05-01,2025-06-01,2025-07-01,2025-08-01,2025-09-01,2025-10-01,2025-11-01,2025-12-01,2026-01-01,2026-02-01


## 3. Slot Release by Release Date

Here we call the slot data by choosing a specific date and print it in a readable format. This is done using the URL:

__/beta/terminal-slots/releases/{date}/__ where __date__ is the release date, in the "YYYY-MM-DD" string format.


We then save the entire dataset as a local variable called `release_df`.

In [6]:
# Function to get the slot releases for a specific date
def get_slot_releases(date):
    uri = urljoin(API_BASE_URL, f'/beta/terminal-slots/releases/{date}/')
    headers = {
            "Authorization": "Bearer {}".format(access_token),
            "accept": "text/csv"
        }

    response = requests.get(uri, headers=headers)

    if response.status_code == 200:
        df = response.content.decode('utf-8')
        df = pd.read_csv(StringIO(df))
        return df

    elif response.content == b'{"errors":[{"code":"object_not_found","detail":"Object not found"}]}':
        print('Bad Date')
        return None
    
    else:
        print('Bad Request')
        return None

### N.B. Plan Limits

__Premium__ Users can choose any release date, as they have full access to the dataset.

__Trial__ Users must choose a release date within the last 2 weeks.

In [8]:
# Calling slot release function
release_df = get_slot_releases("2024-10-22")
release_df.head()

,ReleaseDate,TerminalCode,TerminalName,Total,M+0,M+1,M+2,M+3,M+4,M+5,...,M+4StartDate,M+5StartDate,M+6StartDate,M+7StartDate,M+8StartDate,M+9StartDate,M+10StartDate,M+11StartDate,M+12StartDate,M>12StartDate
0,2024-10-22,adriatic,Adriatic,0,0,0,0,0,0,0,...,2025-02-01,2025-03-01,2025-04-01,2025-05-01,2025-06-01,2025-07-01,2025-08-01,2025-09-01,2025-10-01,2025-11-01
1,2024-10-22,brunsbuttel,Brunsbuttel,0,0,0,0,0,0,0,...,2025-02-01,2025-03-01,2025-04-01,2025-05-01,2025-06-01,2025-07-01,2025-08-01,2025-09-01,2025-10-01,2025-11-01
2,2024-10-22,deutsche-ostsee,Deutsche Ostsee,20,1,3,4,1,1,1,...,2025-02-01,2025-03-01,2025-04-01,2025-05-01,2025-06-01,2025-07-01,2025-08-01,2025-09-01,2025-10-01,2025-11-01
3,2024-10-22,dragon,Dragon,3,1,2,0,0,0,0,...,2025-02-01,2025-03-01,2025-04-01,2025-05-01,2025-06-01,2025-07-01,2025-08-01,2025-09-01,2025-10-01,2025-11-01
4,2024-10-22,eems-energy-terminal,EemsEnergyTerminal,7,1,6,0,0,0,0,...,2025-02-01,2025-03-01,2025-04-01,2025-05-01,2025-06-01,2025-07-01,2025-08-01,2025-09-01,2025-10-01,2025-11-01


## 4. Terminal List

Here we call the list of terminals and their uuid, and print it in a readable format. This is done using the URL:

__'beta/terminal-slots/terminals/'__


We then save the entire dataset as a local variable called `terminal_list`.

In [9]:
# Function to get the list of terminals and their uuids (as well as their start and latest release date)
def get_terminal_list():
    uri = urljoin(API_BASE_URL,'beta/terminal-slots/terminals/')
    headers = {
            "Authorization": "Bearer {}".format(access_token),
            "accept": "text/csv"
        }
    response = requests.get(uri, headers=headers)
    if response.status_code == 200:
        df = response.content.decode('utf-8')
        df = pd.read_csv(StringIO(df))
    else:
        print('Bad Request')
    return df

In [10]:
# Call terminal list function
terminal_list = get_terminal_list()
terminal_list.head()

,TerminalUUID,TerminalCode,TerminalName,firstAvailableRelease,latestAvailableRelease
0,00317185-978a-4df5-970c-2c28d3ab893c,grain-lng,Isle of Grain,2023-11-28,2025-01-14
1,0031994e-f370-4927-ba88-a4e7a78c42db,zeebrugge,Zeebrugge,2023-11-28,2025-01-14
2,00338f3f-8875-435d-87a9-f83d9a5c5241,dunkerque,Dunkerque,2024-11-15,2025-01-14
3,00344dd2-5608-4413-a2f4-c52c747a286a,dragon,Dragon,2023-11-28,2025-01-14
4,003497c6-ed32-412f-95ef-c3b1f962464e,brunsbuttel,Brunsbuttel,2024-05-28,2025-01-14


## Fetching Slots Data specific to one terminal

Now that we can see all the terminal data available to us, we can start to define what terminal we want to call slots data for (by referring to `terminal_list` above).

The first step is to choose which terminal uuid (`my_uuid`), then the request will return all the historical data available for that terminal.

In [11]:
# Function to collect and store historical slots for one specific terminal
def get_individual_terminal(terminal_uuid):
    uri = urljoin(API_BASE_URL, f'/beta/terminal-slots/terminals/{terminal_uuid}/')
    headers = {
            "Authorization": "Bearer {}".format(access_token),
            "accept": "text/csv"
        }
    response = requests.get(uri, headers=headers)
    if response.status_code == 200:
        df = response.content.decode('utf-8')
        df = pd.read_csv(StringIO(df))
        return df

    elif response.content == b'{"errors":[{"code":"object_not_found","detail":"Object not found"}]}':
        print('Bad Terminal Request')
        return None
    else:
        print('Bad Request')
        return None

In [14]:
# Call individual terminal function (Dragon)
my_uuid = terminal_list[terminal_list['TerminalName']=='Dragon']['TerminalUUID'].tolist()[0]
dragon_hist_df = get_individual_terminal(my_uuid)
dragon_hist_df

,TerminalCode,TerminalName,ReleaseDate,Total,M+0,M+1,M+2,M+3,M+4,M+5,...,M+4StartDate,M+5StartDate,M+6StartDate,M+7StartDate,M+8StartDate,M+9StartDate,M+10StartDate,M+11StartDate,M+12StartDate,M>12StartDate
0,dragon,Dragon,2023-11-28,2,0,2,0,0,0,0,...,2024-03-01,2024-04-01,2024-05-01,2024-06-01,2024-07-01,2024-08-01,2024-09-01,2024-10-01,2024-11-01,2024-12-01
1,dragon,Dragon,2023-11-29,2,0,2,0,0,0,0,...,2024-03-01,2024-04-01,2024-05-01,2024-06-01,2024-07-01,2024-08-01,2024-09-01,2024-10-01,2024-11-01,2024-12-01
2,dragon,Dragon,2023-11-30,2,2,0,0,0,0,0,...,2024-04-01,2024-05-01,2024-06-01,2024-07-01,2024-08-01,2024-09-01,2024-10-01,2024-11-01,2024-12-01,2025-01-01
3,dragon,Dragon,2023-12-01,2,2,0,0,0,0,0,...,2024-04-01,2024-05-01,2024-06-01,2024-07-01,2024-08-01,2024-09-01,2024-10-01,2024-11-01,2024-12-01,2025-01-01
4,dragon,Dragon,2023-12-04,1,1,0,0,0,0,0,...,2024-04-01,2024-05-01,2024-06-01,2024-07-01,2024-08-01,2024-09-01,2024-10-01,2024-11-01,2024-12-01,2025-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280,dragon,Dragon,2025-01-08,0,0,0,0,0,0,0,...,2025-05-01,2025-06-01,2025-07-01,2025-08-01,2025-09-01,2025-10-01,2025-11-01,2025-12-01,2026-01-01,2026-02-01
281,dragon,Dragon,2025-01-09,0,0,0,0,0,0,0,...,2025-05-01,2025-06-01,2025-07-01,2025-08-01,2025-09-01,2025-10-01,2025-11-01,2025-12-01,2026-01-01,2026-02-01
282,dragon,Dragon,2025-01-10,0,0,0,0,0,0,0,...,2025-05-01,2025-06-01,2025-07-01,2025-08-01,2025-09-01,2025-10-01,2025-11-01,2025-12-01,2026-01-01,2026-02-01
283,dragon,Dragon,2025-01-13,0,0,0,0,0,0,0,...,2025-05-01,2025-06-01,2025-07-01,2025-08-01,2025-09-01,2025-10-01,2025-11-01,2025-12-01,2026-01-01,2026-02-01


## 5. Historical Slots

Here we collect all the historical slots for each available terminal. This is done using the URL:

__/beta/terminal-slots/terminals/{terminal_uuid}/__

First we define the function that imports the data from the Spark API.

We then call that function, and define the list of terminal uuids.


We save the output as a local variable called `all_terminals_historical`.

### N.B. Plan Limits

__Premium__ Users have full access to the dataset.

__Trial__ Users only have access the latest 2 weeks' worth of datapoints.

In [17]:
# Function to collect and store each terminal's historical slots data
def get_all_terminal_data(terminal_list):
    terminals_all = pd.DataFrame()
    for i in range(len(terminal_list)):
        print(terminal_list['TerminalName'].loc[i])
        terminal_df = get_individual_terminal(terminal_list['TerminalUUID'].loc[i])
        time.sleep(0.1)
        terminals_all = pd.concat([terminals_all,terminal_df])
    return terminals_all

In [18]:
# Calling all terminal data function
all_terminal_historical = get_all_terminal_data(terminal_list)
all_terminal_historical.head()

Isle of Grain
Zeebrugge
Dunkerque
Dragon
Brunsbuttel
OLT Toscana
Le Havre
Piombino
EemsEnergyTerminal
Adriatic
Fos Cavaou
Montoir
Wilhelmshaven 1
South Hook
Inkoo
Deutsche Ostsee
Klaipeda
Gate


,TerminalCode,TerminalName,ReleaseDate,Total,M+0,M+1,M+2,M+3,M+4,M+5,...,M+4StartDate,M+5StartDate,M+6StartDate,M+7StartDate,M+8StartDate,M+9StartDate,M+10StartDate,M+11StartDate,M+12StartDate,M>12StartDate
0,grain-lng,Isle of Grain,2023-11-28,1,0,1,0,0,0,0,...,2024-03-01,2024-04-01,2024-05-01,2024-06-01,2024-07-01,2024-08-01,2024-09-01,2024-10-01,2024-11-01,2024-12-01
1,grain-lng,Isle of Grain,2023-11-29,1,0,1,0,0,0,0,...,2024-03-01,2024-04-01,2024-05-01,2024-06-01,2024-07-01,2024-08-01,2024-09-01,2024-10-01,2024-11-01,2024-12-01
2,grain-lng,Isle of Grain,2023-11-30,2,2,0,0,0,0,0,...,2024-04-01,2024-05-01,2024-06-01,2024-07-01,2024-08-01,2024-09-01,2024-10-01,2024-11-01,2024-12-01,2025-01-01
3,grain-lng,Isle of Grain,2023-12-01,2,2,0,0,0,0,0,...,2024-04-01,2024-05-01,2024-06-01,2024-07-01,2024-08-01,2024-09-01,2024-10-01,2024-11-01,2024-12-01,2025-01-01
4,grain-lng,Isle of Grain,2023-12-04,2,2,0,0,0,0,0,...,2024-04-01,2024-05-01,2024-06-01,2024-07-01,2024-08-01,2024-09-01,2024-10-01,2024-11-01,2024-12-01,2025-01-01


# Analytics Gallery
Want to gain market insights using our data?

Take a look at our [Analytics Gallery](https://www.sparkcommodities.com/api/code-examples/analytics-examples.html) on the Spark API website, which includes

- __Regas Slots vs US Arb__ - See how the US Arb (M+1, M+2, M+3) correlates with monthly average slots for all European Terminals.
__Note: Requires Cargo and Access subscriptions__


Take a look at our Regas Slots vs US Arb chart [here](https://www.sparkcommodities.com/api/code-examples/analytics-examples.html). 